# Setup

## Libraries

In [10]:
import os
import json
import pandas as pd
import traceback
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv
import PyPDF2

## API keys

In [11]:
load_dotenv() # Take enviroment variables from .env
KEY=os.getenv("OPENAI_API_KEY")

In [12]:
llm=ChatOpenAI(openai_api_key=KEY, model_name='gpt-3.5-turbo', temperature=0.5)

# Templates

### Response Code (Template)

In [13]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

### 1st Chain - Create a Quiz


In [36]:
TEMPLATE ="""
Text:{text1}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [37]:
quiz_prompt = PromptTemplate(
    input_variables=["text1", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
)

In [38]:
quiz_chain=LLMChain(llm=llm, prompt= quiz_prompt, output_key='quiz', verbose= True)

### 2nd Chain - Evaluate the complexity

In [39]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis.
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [40]:
quiz_evaluation_prompt = PromptTemplate(input_variables=['subject', 'quiz'], template=TEMPLATE)

In [41]:
review_chain = LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key='review', verbose= True)

### Sequential Chain (two chain mix)

In [42]:
seq_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text1", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)


# Data

### Get Data

In [43]:
filepath = r'C:\Users\Tomy\Documents\iNeuron-GenerativeAI-Proyect01\data.txt'

In [44]:
with open(filepath, 'r') as f:
    TEXT = f.read()

### Serialize the Python dictonary into a JSON formatted string

In [45]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [46]:
NUMBER=5
SUBJECT="biology"
TONE="simple"

## Token Setup

### Traking OpenAI Token

In [47]:
#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=seq_chain(
        {
            "text1": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )

C:\Users\Tomy\Documents\iNeuron-GenerativeAI-Proyect01\env\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:The academic discipline of artificial intelligence was established at a research workshop held at Dartmouth College in 1956 and has experienced several waves of advancement and optimism in the decades since.[20] Since its inception, researchers in the field have raised philosophical and ethical arguments about the nature of the human mind and the consequences of creating artificial beings with human-like intelligence; these issues have previously been explored by myth, fiction and philosophy since antiquity.[21] The concept of automated art dates back at least to the automata of ancient Greek civilization, where inventors such as Daedalus and Hero of Alexandria were described as having designed machines capable of writing text, generating sounds, and playing music.[22][23] The tradition of creative automatons has flourished throughout history, exemplified by Maillardet's automato

### Cost of OpenAI

In [48]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:3258
Prompt Tokens:2312
Completion Tokens:946
Total Cost:0.005359999999999999


In [55]:
quiz = response.get('quiz')
quiz = json.loads(quiz)
quiz

{'1': {'mcq': 'What was the groundbreaking 1950 paper by Alan Turing that posed fundamental questions about machine reasoning similar to human intelligence?',
  'options': {'a': 'Computing Machinery and Intelligence',
   'b': 'Artificial Intelligence: A New Era',
   'c': 'The Rise of Machines',
   'd': 'The Future of Robotics'},
  'correct': 'a'},
 '2': {'mcq': 'Which ancient Greek inventors were described as having designed machines capable of writing text, generating sounds, and playing music?',
  'options': {'a': 'Daedalus and Hero of Alexandria',
   'b': 'Archimedes and Aristotle',
   'c': 'Hippocrates and Socrates',
   'd': 'Plato and Pythagoras'},
  'correct': 'a'},
 '3': {'mcq': 'In which year did the emergence of deep learning drive progress and research in image classification, speech recognition, and natural language processing?',
  'options': {'a': '2000', 'b': '2010', 'c': '2005', 'd': '2015'},
  'correct': 'd'},
 '4': {'mcq': 'Which neural network model enabled advancement

In [54]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [60]:
quiz = pd.DataFrame(quiz_table_data)
quiz.to_csv('AGI-Quiz.csv', index=False)

In [61]:
quiz

,MCQ,Choices,Correct
0,What was the groundbreaking 1950 paper by Alan...,a: Computing Machinery and Intelligence | b: A...,a
1,Which ancient Greek inventors were described a...,a: Daedalus and Hero of Alexandria | b: Archim...,a
2,In which year did the emergence of deep learni...,a: 2000 | b: 2010 | c: 2005 | d: 2015,d
3,Which neural network model enabled advancement...,a: Transformer network | b: GPT-1 | c: GPT-2 |...,a
4,What AI model released in 2021 marked the emer...,a: GPT-4 | b: ImageBind | c: Midjourney | d: S...,b
